# MIDI - Music

In [1]:
import numpy as np
import mido
from mido import Message, MidiFile, MetaMessage
import time

In [2]:
names = "c c# d d# e f f# g g# a a# h C C# D D# E F F# G G# A A# H".split(' ')
midi_map = dict(zip(names,[v for v in range(60,85)]))
midi_map['r']=0



def get_midi_port_names():
    return mido.get_output_names()
    
def get_midi_port(number):
    port = get_midi_port_names()[number]
    return mido.open_output(port)

def play_midi_file(name):
    midi = MidiFile(name)
    midi.print_tracks()
    with get_midi_port(1) as out:
        for msg in midi.play():
            out.send(msg)         
            
###########################            
            
def create_header():
    header = []
    header.append(MetaMessage('track_name', name='Klavier', time=0))
    header.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
    header.append(MetaMessage('key_signature', key='C', time=0))
    header.append(MetaMessage('set_tempo', tempo=500000, time=0))
    return header
    
def save_midi_song(filename,messages):
    with open(filename, "wb") as file:
        for message in messages:
            print(message.bytes())
            bytes = bytearray(message.bytes())              
            file.write(bytes)            

########### Tunes abspielen ################

DEFAULT_TEMPO = 500000
DEFAULT_TICKS_PER_BEAT = 480

    
def play_tune(tune):
    '''
    Spiel eine Song in Form eines formatierten Strings.
    '''
    messages = parse_tune(tune)
    play_messages(messages)
    
    
def play_messages(messages):    
    '''
    Spielt eine Sequenz von MIDI-Messages ab.
    '''
    with get_midi_port(1) as out:
        for msg in get_messages(messages):
            out.send(msg)
            
def get_messages(messages):
    tempo = DEFAULT_TEMPO
    ticks_per_beat=DEFAULT_TICKS_PER_BEAT
    for msg in get_next_message(messages):
        if msg.time > 0:
            delta = tick2second(msg.time, ticks_per_beat, tempo)
        else:
            delta = 0

        yield msg.copy(time=delta)

        if msg.type == 'set_tempo':
            tempo = msg.tempo     
            

def tick2second(tick, ticks_per_beat, tempo):
    scale = tempo * 1e-6 / ticks_per_beat
    return tick * scale

    
def get_next_message(messages,now=time.time):   
    start_time = now()
    input_time = 0.0

    for msg in messages:
        input_time += msg.time
        playback_time = now() - start_time
        duration_to_next_event = input_time - playback_time

        if duration_to_next_event > 0.0:
            time.sleep(duration_to_next_event)

        if isinstance(msg, MetaMessage):
            continue
        else:
            yield msg
    return
      
          

#############################

ONE_TICK = 0.001

def parse_tune(tune):
    '''
    Liest einen String ein und erstellt daraus eine Liste von MIDI-Messages.
    Erstellt aus einer Note je zwei MIDI-Messages zum Start und Stop des Klangs. 
    Ein optionaler Header wird vorangestellt.
    Die Tonhöhe wird wird in der Notation C oder f# angegeben. 
    Der Notenwert wird mit Zahlen 1, 2, 4, 8 usw. für ganze, halbe, Viertel- bzw Achtelnoten.    
    '''
    tune = tune.replace('|','').split(' ')  
    tune = [note.strip() for note in tune if (len(note.strip()) > 1)]  
    notes = [(note[0:-1],float(note[-1])) for note in tune]
     
    messages = [] #create_header()
        
    for name,value in notes:
        volume = 0 if name=='r' else 80
    
        messages.append(Message('note_on', note=midi_map[name],velocity=volume,time=ONE_TICK) )   
        messages.append(Message('note_on', note=midi_map[name],velocity=0,time=2 / value - ONE_TICK) )

    return messages

haenschen_klein = "g4 e4 e4 r4   | f4 d4 d4 r4  |  || c4 d4 e4 f4 | g4 g4 g4 r4"
#print(parse_tune(haenschen_klein) )

In [3]:
haenschen_klein = "g4 e4 e4 r4   | f4 d4 d4 r4  |  || c4 d4 e4 f4 | g4 g4 g4 r4"
play_tune(haenschen_klein) 

In [4]:
mid = mido.MidiFile('midi/Heiko.mid')
with get_midi_port(1) as port:
    for msg in mid.play():
       # print(msg)
        port.send(msg)   

In [5]:
mid = mido.MidiFile('Heiko1.mid')
print(mid)

FileNotFoundError: [Errno 2] No such file or directory: 'Heiko1.mid'

In [ ]:
jingle_bells = "e4 e4 e2 | e4 e4 e2  | e4 g4 c4 d4 | e1 | f4 f4 f4 f4 |  f4 e4 e2 | e4 d4 d4 e4 | d2 g2 |  e4 e4 e2 | e4 e4 e2 | e4 g4 c4 d4 | e1 | f4 f4 f4 f4 | f4 e4 e4 e4 | g4 g4 f4 d4 | c1"

play_tune(jingle_bells)

In [ ]:
sound_of_silence = "  r4 d8 d8 f8 f8 a8 a8|  g1 | r8 c8 c8 c8 e8 e8 g8 g8 |  f1 | r8 f8 f8 f8 a8 a8 C8 C8 | D4 D8 C8  C2 | \
r4 f8 f8 a8 a8 C8 C8 | "
play_tune(sound_of_silence)